# Import Libraries


In [7]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore  
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from dotenv import load_dotenv
import os
load_dotenv()

True

# Extract text from PDF files

In [ ]:

def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    
    documents = loader.load()
    return documents

In [9]:
extracted_data=load_pdf_files("../data")

In [10]:
extracted_data[10]

Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': '..\\data\\Medical_book.pdf', 'total_pages': 637, 'page': 10, 'page_label': '11'}, page_content='Rhonda Cloos, R.N.\nMedical Writer\nAustin, TX\nGloria Cooksey, C.N.E\nMedical Writer\nSacramento, CA\nAmy Cooper, M.A., M.S.I.\nMedical Writer\nVermillion, SD\nDavid A. Cramer, M.D.\nMedical Writer\nChicago, IL\nEsther Csapo Rastega, R.N., B.S.N.\nMedical Writer\nHolbrook, MA\nArnold Cua, M.D.\nPhysician\nBrooklyn, NY\nTish Davidson, A.M.\nMedical Writer\nFremont, California\nDominic De Bellis, Ph.D.\nMedical Writer/Editor\nMahopac, NY\nLori De Milto\nMedical Writer\nSicklerville, NJ\nRobert S. Dinsmoor\nMedical Writer\nSouth Hamilton, MA\nStephanie Dionne, B.S.\nMedical Writer\nAnn Arbor, MI\nMartin W. Dodge, Ph.D.\nTechnical Writer/Editor\nCentinela Hospital and Medical\nCenter\nInglewood, CA\nDavid Doermann\nMedical

In [ ]:
len(extracted_data)

637

# filter_to_minimal_documents

In [11]:
from typing import List
from langchain.schema import Document
def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of Document objects, return a new list of Document objects
    containing only 'source' in metadata and the original page_content.
    """
    minimal_docs: list[Document] = []
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src},
            )
        )
    
    return minimal_docs

In [12]:
minimal_docs=filter_to_minimal_docs(extracted_data)


# Split the documents into smaller chunks

In [ ]:

def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20
    )
    texts_chunks= text_splitter.split_documents(minimal_docs)
    return texts_chunks

In [14]:
texts_chunks = text_split(minimal_docs)
print(f"Number of chunks:{len(texts_chunks)}")

Number of chunks:5859


# Vector Embeddings

In [15]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings


In [16]:
embeddings = download_hugging_face_embeddings()

C:\Users\admin\AppData\Local\Temp\ipykernel_6324\1001347487.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [17]:
query_text = "What is a common allergy?"
embedding_vector = embeddings.embed_query(query_text)

In [18]:
print("Sample of the embedding vector (first 20 numbers):")
print(embedding_vector[:20])

Sample of the embedding vector (first 20 numbers):
[0.04702260345220566, -0.006634933408349752, -0.01139666885137558, -0.031360842287540436, 0.033007990568876266, 0.03826579824090004, 0.07354530692100525, 0.08192341774702072, -0.08451011031866074, 0.027469314634799957, 0.0592004731297493, -0.06414079666137695, -0.026036029681563377, 0.05833462253212929, -0.04474414139986038, 0.06623373180627823, -0.011161021888256073, -0.023217888548970222, 0.014017648063600063, -0.05721267685294151]


# Storing Embeddings in Pinecone

In [19]:

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


In [20]:
index_name = "medical-chatbot"

In [ ]:
vector_store = PineconeVectorStore.from_documents(
    texts_chunks,
    embeddings,
    index_name=index_name
)

In [21]:
print("Connecting to existing index in Pinecone...")
vector_store = PineconeVectorStore.from_existing_index(
    index_name,
    embeddings
)
print("Connection successful!")

Connecting to existing index in Pinecone...
Connection successful!


# Similarity Search & Retrieval

In [22]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [23]:
retrieved_docs = retriever.invoke("What is Acne?")
retrieved_docs

[Document(id='1811b710-fbe5-43b2-8394-d83e0536b4c5', metadata={'source': '..\\data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='d4a1434f-3129-432c-a390-1e9117f54591', metadata={'source': '..\\data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'),
 Document(id='ea02179f-578f-49a5-b56d-a4bcaa38bdc4', metadata={'source': '..\\data\\Medical_book.pdf'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin becom

In [24]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
from langchain_openai import ChatOpenAI
chatModel = ChatOpenAI(model="gpt-4o")

In [25]:
system_prompt = (
    "You are an Medical assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [28]:
question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# TESTING

In [ ]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

Acromegaly is a disorder caused by the abnormal release of chemicals from the pituitary gland, leading to increased growth in bones and soft tissues. It occurs after bone growth has stopped. When this abnormality happens before growth stops, it results in gigantism, characterized by unusual height


In [ ]:
response = rag_chain.invoke({"input": "what is Acne?"})
print(response["answer"])

The treatment of acne depends on its severity. For mild noninflammatory acne, topical treatments such as tretinoin, benzoyl peroxide, adapalene, or salicylic acid are recommended. For inflammatory acne, additional methods like topical antibiotics may be used, while severe cases may require treatments like isotretinoin.
